In [110]:
# To enable plotting graphs in Jupyter notebook
%matplotlib inline
# Enable auto complete
%config Completer.use_jedi = False

In [111]:
import pandas as pd
import numpy as np

# Import ploting modules
import matplotlib.pyplot as plt
import seaborn as sns

# Import model modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.compose import ColumnTransformer

In [112]:
des_df = pd.read_csv('LCDataDictionary.csv')

In [113]:
des_df = des_df[des_df.Description.notna()]
des_df = des_df[des_df.LoanStatNew.notna()]

In [114]:
data = pd.read_csv('lending_csv_v3.zip', compression='zip')
data.head()

/Users/vikasg-l/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (49,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1604328,70811807,NaN,12800.0,12800.0,12800.0,60 months,21.18,347.59,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,241274,982256,NaN,2000.0,2000.0,2000.0,36 months,12.69,67.09,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,1365555,77088678,NaN,29075.0,29075.0,29075.0,36 months,27.34,1192.31,F,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,47386,31347403,NaN,16000.0,16000.0,16000.0,36 months,8.39,504.27,A,...,NaN,NaN,Cash,Y,Sep-2017,COMPLETE,Aug-2017,2041.57,45.0,24.0
4,910540,4287047,NaN,35000.0,35000.0,35000.0,36 months,18.49,1273.96,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
df.num_op_rev_tl = df.num_op_rev_tl.fillna(df.num_op_rev_tl.mean()).astype(int)

In [342]:
# [
#  'next_pymnt_d',
#  'mths_since_rcnt_il',
#  'all_util',





#  'open_il_24m',
#  'open_il_12m',

# ]

# [  
#  'chargeoff_within_12_mths', 
#                    'collection_recovery_fee',
#                   ]

for col in df.isna().sum().sort_values(ascending=False).index:
    if df[col].isna().sum()> 0:
        print(col)
#         print('1:- ',des_df[des_df.LoanStatNew == col])
        print('2:- ',data[col].isna().sum())
#         print('3:- \n', data[col].value_counts())
#         print(data[col].describe())
        print('\n\n')

num_bc_tl
2:-  15119



tot_coll_amt
2:-  15119



mo_sin_old_rev_tl_op
2:-  15119



mo_sin_rcnt_rev_tl_op
2:-  15119



mo_sin_rcnt_tl
2:-  15119



total_rev_hi_lim
2:-  15119



num_accts_ever_120_pd
2:-  15119



num_actv_bc_tl
2:-  15119



num_actv_rev_tl
2:-  15119



num_il_tl
2:-  15119



num_rev_accts
2:-  15119



num_rev_tl_bal_gt_0
2:-  15119



tot_cur_bal
2:-  15119



tot_hi_cred_lim
2:-  15119



num_tl_90g_dpd_24m
2:-  15119



num_tl_op_past_12m
2:-  15119



total_il_high_credit_limit
2:-  15119



num_tl_30dpd
2:-  15119



bc_util
2:-  13437



percent_bc_gt_75
2:-  13360



bc_open_to_buy
2:-  13278



mths_since_recent_bc
2:-  13098



num_sats
2:-  12321



num_bc_sats
2:-  12321



acc_open_past_24mths
2:-  10345



total_bc_limit
2:-  10345



total_bal_ex_mort
2:-  10345



mort_acc
2:-  10345



last_pymnt_d
2:-  906



revol_util
2:-  166



pub_rec_bankruptcies
2:-  153



dti
2:-  52



last_credit_pull_d
2:-  15



collections_12_mths_ex_med
2:-  14



In [316]:
missing_col = set(data.columns) - set(des_df.LoanStatNew)
missing_col_df = data.loc[:, missing_col]
invalid_value_col = list(missing_col_df.columns[missing_col_df.isna().sum() > 240000])
df = data.drop(invalid_value_col + [
    'member_id', 'id', 'Unnamed: 0', 'pymnt_plan', 'zip_code', 'next_pymnt_d', 'acc_now_delinq','policy_code','application_type',
    'disbursement_method', 'dti_joint', 'annual_inc_joint', 'desc', 'next_pymnt_d', 'mths_since_rcnt_il', 'all_util', 'max_bal_bc',
    'open_rv_24m', 'total_cu_tl', 'total_bal_il', 'open_il_24m', 'open_il_12m', 'disbursement_method', 'hardship_flag', 
    'collection_recovery_fee', 'il_util', 'open_act_il', 'title', 'emp_title','chargeoff_within_12_mths'
], axis=1)
df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,debt_settlement_flag
0,12800.0,12800.0,12800.0,60 months,21.18,347.59,E,E3,10+ years,MORTGAGE,...,3.0,82.9,50.0,0.0,0.0,337444.0,51847.0,2100.0,31884.0,N
1,2000.0,2000.0,2000.0,36 months,12.69,67.09,B,B5,10+ years,MORTGAGE,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,N
2,29075.0,29075.0,29075.0,36 months,27.34,1192.31,F,F5,4 years,MORTGAGE,...,4.0,89.5,50.0,0.0,0.0,164709.0,48895.0,6300.0,55084.0,N
3,16000.0,16000.0,16000.0,36 months,8.39,504.27,A,A5,10+ years,MORTGAGE,...,1.0,89.5,66.7,0.0,0.0,54454.0,32271.0,9700.0,39054.0,Y
4,35000.0,35000.0,35000.0,36 months,18.49,1273.96,D,D2,10+ years,MORTGAGE,...,0.0,66.0,25.0,0.0,0.0,1412518.0,336990.0,13100.0,438745.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,22000.0,22000.0,22000.0,60 months,24.99,645.61,F,F3,7 years,RENT,...,6.0,88.9,66.7,1.0,0.0,73442.0,54556.0,5200.0,58042.0,N
299996,6500.0,6500.0,6500.0,36 months,8.18,204.23,B,B1,< 1 year,RENT,...,1.0,46.7,66.7,0.0,0.0,61571.0,50635.0,8400.0,53171.0,N
299997,30000.0,30000.0,30000.0,36 months,18.75,1095.90,D,D3,3 years,RENT,...,0.0,94.0,40.0,0.0,0.0,125128.0,40566.0,44300.0,68838.0,N
299998,21125.0,21125.0,21125.0,36 months,19.19,776.39,E,E3,10+ years,RENT,...,8.0,97.4,60.0,0.0,0.0,197663.0,43565.0,28200.0,30417.0,N


In [325]:
df.num_op_rev_tl.mode()

0    6.0
dtype: float64

In [328]:
# df.mths_since_last_major_derog = df.mths_since_last_major_derog.fillna(0)
# df.mths_since_last_record = df.mths_since_last_record.fillna(0)
# df.mths_since_recent_bc_dlq = df.mths_since_recent_bc_dlq.fillna(0)
# df.mths_since_recent_revol_delinq = df.mths_since_recent_revol_delinq.fillna(0)
# df.mths_since_last_delinq = df.mths_since_last_delinq.fillna(0)
# df.open_acc_6m = df.open_acc_6m.fillna(0.0).astype(int)
# df.inq_fi = df.inq_fi.fillna(0.0).astype(int)
# df.inq_last_12m = df.inq_last_12m.fillna(0.0).astype(int)
# df.open_rv_12m = df.open_rv_12m.fillna(0.0).astype(int)
# df.open_rv_12m = df.open_rv_12m.fillna(0.0).astype(int)
# df.mths_since_recent_inq = df.mths_since_recent_inq.fillna(0).astype(int)
# df.num_tl_120dpd_2m = df.num_tl_120dpd_2m.fillna(0).astype(int)
# df.mo_sin_old_il_acct = df.mo_sin_old_il_acct.fillna(df.mo_sin_old_il_acct.mean())
# df.term = df.term.str.replace(' months', '').astype(int)
# df.emp_length = df.emp_length.fillna(df.emp_length.mode()).astype(str)
# df.pct_tl_nvr_dlq = df.pct_tl_nvr_dlq.fillna(100).astype(int)
# df.avg_cur_bal = df.avg_cur_bal.fillna(df.avg_cur_bal.median()).astype(int)
df.num_op_rev_tl = df.num_op_rev_tl.fillna(df.num_op_rev_tl.mean()).astype(int)


In [273]:
df.isna().sum().sort_values(ascending=False)

mo_sin_old_il_acct     23632
emp_title              19664
emp_length             18079
pct_tl_nvr_dlq         15158
avg_cur_bal            15123
                       ...  
out_prncp_inv              0
out_prncp                  0
initial_list_status        0
total_acc                  0
loan_amnt                  0
Length: 92, dtype: int64

# Converted loan term data type from string to number

## Filling column value with default 0 as per assumption it should be 0 month

## Verify column with nan value more than 60%

In [240]:
col_with_missing_values= [col for col in df.loc[:,df.columns[df.isna().sum() > 200000]].columns if col in set(des_df.LoanStatNew)]

In [241]:
df = df.drop(col_with_missing_values, axis=1)

In [242]:
len(df.columns[df.isna().sum() > 0])

48

In [243]:
df.columns[df.isna().sum()> 150000]
des_df.LoanStatNew.isin(df.loc[:,df.columns[df.isna().sum() > 150000]].columns)
des_df.loc[: , des_df.columns.isin(df.loc[:,df.columns[df.isna().sum() > 150000]].columns)]

""
0
1
2
3
4
...
110
111
112
113


In [122]:
for col in df.select_dtypes(exclude='object').columns:
    df[col].fillna(df[col].mean(), inplace=True)
for col in df.select_dtypes(include='object').columns:
    df[col].fillna(df[col].mode(), inplace=True)


In [123]:
converted_df = df.sample(frac=0.05, random_state=1)
converted_df_x = converted_df.drop('loan_status', axis=1)

In [124]:
def rfe(X, y):
        '''
        Returns a dataframe with only a subset of features/columns retained
        '''
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.feature_selection import RFE
        est = RandomForestClassifier(n_estimators=100, oob_score=True, max_features=5)
#        selectK = SelectKBest(score_func = f_classif, k=45)
        select_rfe = RFE(estimator=est, n_features_to_select=50, step=0.15)
        select_k=select_rfe
        
        select_k.fit(X,y)
        return select_k
#         K_featnames = feature_names[selectK_mask]
#         print ("reduced RFE features:")
#         print(K_featnames)
#         reduced_df = pd.read_csv(filename, index_col=0)
#         reduced_df = reduced_df[reduced_df.columns[selectK_mask]]

#         return reduced_df
selected_k = rfe(converted_df_x,converted_df.loan_status)


ValueError: could not convert string to float: 'B'

In [125]:
converted_df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
205179,12000.0,12000.0,12000.0,36,13.11,404.97,B,B4,Industrial Furnace Company,10+ years,...,16.7,0.0,0.0,168163.222465,40713.0,29442.0,42421.431289,N,Cash,N
101710,24000.0,24000.0,24000.0,60,13.33,550.14,C,C3,Engineer,3 years,...,100.0,0.0,0.0,73536.000000,55008.0,48600.0,23236.000000,N,Cash,N
89106,14025.0,14025.0,14025.0,36,13.99,479.28,C,C4,General manager,10+ years,...,25.0,0.0,0.0,101103.000000,27003.0,18000.0,19320.000000,N,Cash,N
193046,2400.0,2400.0,2400.0,36,15.31,83.57,C,C5,Paralegal,10+ years,...,100.0,0.0,0.0,77164.000000,65043.0,4200.0,63664.000000,N,Cash,N
99093,21000.0,21000.0,21000.0,36,7.90,657.10,A,A4,Maintenance,10+ years,...,66.7,0.0,0.0,106561.000000,33343.0,34200.0,17361.000000,N,Cash,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6205,7700.0,7700.0,7700.0,36,15.61,269.23,D,D1,Golf maintenance,10+ years,...,36.4,1.0,0.0,29500.000000,20423.0,15000.0,12000.000000,N,Cash,N
195953,18000.0,18000.0,18000.0,36,9.75,578.70,B,B3,Supervisor,8 years,...,40.0,0.0,0.0,204624.000000,60742.0,24800.0,47124.000000,N,Cash,N
189576,20000.0,20000.0,20000.0,60,11.44,439.25,B,B4,NaN,NaN,...,66.7,0.0,0.0,136148.000000,45098.0,34300.0,101848.000000,N,Cash,N
176839,10000.0,10000.0,10000.0,36,12.62,335.12,C,C1,Office Manager,9 years,...,50.0,0.0,0.0,46701.000000,18545.0,10300.0,35301.000000,N,Cash,N


## Trying with Automatically Select the Number of Features 

In [126]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

In [127]:
X = converted_df_x
y = converted_df.loan_status
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)


In [128]:
X.select_dtypes(include='object').columns

Index(['grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership',
       'verification_status', 'issue_d', 'purpose', 'title', 'addr_state',
       'earliest_cr_line', 'initial_list_status', 'last_pymnt_d',
       'last_credit_pull_d', 'application_type', 'hardship_flag',
       'disbursement_method', 'debt_settlement_flag'],
      dtype='object')

# Label Encoding

In [ ]:
cleanup_nums = {
    "grade": {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7},
    "sub_grade": {"A1": 1, "A2": 2, "A3": 3, "A4": 4, "A5": 5, "B1": 6, "B2": 7, "B3": 8, "B4": 9, "B5": 10, "C1": 11, "C2": 12, "C3": 13, "C4": 14, "C5": 15, "D1": 16, "D2": 17, "D3": 18, "D4": 19, "D5": 20, "E1": 21, "E2": 22, "E3": 23, "E4": 24, "E5": 25, "F1": 26, "F2": 27, "F3": 28, "F4": 29, "F5": 30, "G1": 31, "G2": 32, "G3": 33, "G4": 34, "G5": 35}
}

In [148]:
import json
json.dumps({k:i+1 for i,k in enumerate(list(X.sub_grade.sort_values().unique()))})

'{"A1": 1, "A2": 2, "A3": 3, "A4": 4, "A5": 5, "B1": 6, "B2": 7, "B3": 8, "B4": 9, "B5": 10, "C1": 11, "C2": 12, "C3": 13, "C4": 14, "C5": 15, "D1": 16, "D2": 17, "D3": 18, "D4": 19, "D5": 20, "E1": 21, "E2": 22, "E3": 23, "E4": 24, "E5": 25, "F1": 26, "F2": 27, "F3": 28, "F4": 29, "F5": 30, "G1": 31, "G2": 32, "G3": 33, "G4": 34, "G5": 35}'

In [152]:
# X.earliest_cr_line.unique()
# des_df[des_df.LoanStatNew=='earliest_cr_line'].Description.values

X.verification_status.unique()
# des_df[des_df.LoanStatNew=='initial_list_status'].Description.values
# X = X.drop('next_pymnt_d', axis=1)

array(['Not Verified', 'Source Verified', 'Verified'], dtype=object)

In [29]:
# create pipeline
model = SVR(kernel='linear')
rfe = RFECV(model, step=5, cv=cv, scoring='accuracy', n_jobs=5, min_features_to_select = 50)

In [30]:
rfe.fit(X, y)
print('Optimal number of features: {}'.format(rfecv.n_features_))
# model = DecisionTreeClassifier()
# pipeline = Pipeline(steps=[('s',rfe),('m',model)])

# n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=4, error_score='raise')
# report performance
# print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

ValueError: could not convert string to float: 'Fully Paid'

## Deleteing columns who have more than 50% null value

In [71]:
df = data.loc[:, list(data.columns[data.isna().sum() < 150000])]
df.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,1604328,70811807,12800.0,12800.0,12800.0,60 months,21.18,347.59,E,E3,...,50.0,0.0,0.0,337444.0,51847.0,2100.0,31884.0,N,Cash,N
1,241274,982256,2000.0,2000.0,2000.0,36 months,12.69,67.09,B,B5,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,N,Cash,N
2,1365555,77088678,29075.0,29075.0,29075.0,36 months,27.34,1192.31,F,F5,...,50.0,0.0,0.0,164709.0,48895.0,6300.0,55084.0,N,Cash,N
3,47386,31347403,16000.0,16000.0,16000.0,36 months,8.39,504.27,A,A5,...,66.7,0.0,0.0,54454.0,32271.0,9700.0,39054.0,N,Cash,Y
4,910540,4287047,35000.0,35000.0,35000.0,36 months,18.49,1273.96,D,D2,...,25.0,0.0,0.0,1412518.0,336990.0,13100.0,438745.0,N,Cash,N


In [72]:
df = df.drop(['Unnamed: 0', 'id'], axis=1)
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,12800.0,12800.0,12800.0,60 months,21.18,347.59,E,E3,HR Generalis,10+ years,...,50.0,0.0,0.0,337444.0,51847.0,2100.0,31884.0,N,Cash,N
1,2000.0,2000.0,2000.0,36 months,12.69,67.09,B,B5,Kmart,10+ years,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,N,Cash,N
2,29075.0,29075.0,29075.0,36 months,27.34,1192.31,F,F5,Professor,4 years,...,50.0,0.0,0.0,164709.0,48895.0,6300.0,55084.0,N,Cash,N
3,16000.0,16000.0,16000.0,36 months,8.39,504.27,A,A5,Trainng Specialist,10+ years,...,66.7,0.0,0.0,54454.0,32271.0,9700.0,39054.0,N,Cash,Y
4,35000.0,35000.0,35000.0,36 months,18.49,1273.96,D,D2,"Stat medical x-ray tubes, inc.",10+ years,...,25.0,0.0,0.0,1412518.0,336990.0,13100.0,438745.0,N,Cash,N


In [73]:
set(df.select_dtypes(exclude='object').columns)

{'acc_now_delinq',
 'acc_open_past_24mths',
 'annual_inc',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'collection_recovery_fee',
 'collections_12_mths_ex_med',
 'delinq_2yrs',
 'delinq_amnt',
 'dti',
 'fico_range_high',
 'fico_range_low',
 'funded_amnt',
 'funded_amnt_inv',
 'inq_last_6mths',
 'installment',
 'int_rate',
 'last_fico_range_high',
 'last_fico_range_low',
 'last_pymnt_amnt',
 'loan_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_last_delinq',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'open_acc',
 'out_prncp',
 'out_prncp_inv',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'policy_code',
 'pub_rec

In [53]:
cln = df.select_dtypes(exclude='object').columns
for cl in cln:
    sns.boxplot(df.chd, df[cl])
    plt.title(f'Column({cl}) vs chd')
    plt.show()

AttributeError: 'DataFrame' object has no attribute 'chd'